<a href="https://colab.research.google.com/github/dmswl0707/RecomendationSystem_Python/blob/main/01_content_based_filtering_%EC%BB%A8%ED%85%90%EC%B8%A0%EA%B8%B0%EB%B0%98%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/RCS_data/01data_2.csv')
data.head(2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,807,Crime Mystery Thriller,8.1,5915.0,18.457430,Se7en,self-fulfilling prophecy detective s.w.a.t. dr...,Two homicide detectives are on a desperate hun...,7.838691
1,629,Drama Crime Thriller,8.1,3334.0,16.302466,The Usual Suspects,law relatives theft criminal criminal mastermi...,"Held in an L.A. interrogation room, Verbal Kin...",7.700533


#콘텐츠 기반 필터링 추천
컨텐츠 기반 필터링을 이용하여 비슷한 장르를 찾아 추천한다

1. 코사인 유사도를 이용해 장르가 비슷한 영화를 추천 🌟🌟🌟
2. vote_count가 높은 것을 이용하여 추천 🌟🌟🌟

###문자열을 숫자로 벡터화

In [ ]:
data.genres.head(2)

0    Crime Mystery Thriller
1      Drama Crime Thriller
Name: genres, dtype: object

## scikit-learn 문서 전처리 기능
- DictVectorizer : 각 단어의 수를 세어놓은 사전에서 BOW 인코딩 벡터를 만든다.(단어의 사용 빈도)
- CountVectorizer : 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩 벡터를 만든다.(토근 단위의 출현 빈도)
- TfidfVectorizer: CountVectorizer와 비슷하지만 TF-IDF 방식으로 단어의 가중치를 조정한 BOW 인코딩 벡터를 만든다.
- HashingVectorizer : 해시 함수(hash function)을 사용하여 적은 메모리와 빠른 속도로 BOW 인코딩 벡터를 만든다.

### TF - IDF 
TF-IDF(Term Frequency – Inverse Document Frequency) 인코딩은 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어의 경우 문서 구별 능력이 떨어진다고 보아 가중치를 축소하는 방법이다. 
   ### tf-idf(𝑑,𝑡)=tf(𝑑,𝑡)⋅idf(𝑡)
- tf(𝑑,𝑡) : term frequency. 특정한 단어의 빈도수

- idf(𝑡) : inverse document frequency. 특정한 단어가 들어 있는 문서의 수에 반비례하는 수
- 𝑛 : 전체 문서의 수

- df(𝑡): 단어 𝑡를 가진 문서의 수


### CountVectorizer

```
corpus = ['This is the first document']
vect = CountVectorizer(인수)
vect.fit(corpus)
vect.transforms(corpus).toarray()
```

출처 : https://datascienceschool.net/03%20machine%20learning/03.01.03%20Scikit-Learn%EC%9D%98%20%EB%AC%B8%EC%84%9C%20%EC%A0%84%EC%B2%98%EB%A6%AC%20%EA%B8%B0%EB%8A%A5.html




In [ ]:
count_vector = CountVectorizer(ngram_range=(1, 3))
c_vector_genres = count_vector.fit_transform(data['genres'])
c_vector_genres.shape

(485, 394)

In [ ]:
# 단어를 벡터화하여 저장
print(c_vector_genres)

  (0, 152)	1
  (0, 328)	1
  (0, 363)	1
  (0, 167)	1
  (0, 337)	1
  (0, 169)	1
  (1, 152)	1
  (1, 363)	1
  (1, 174)	1
  (1, 185)	1
  (1, 172)	1
  (1, 186)	1
  (2, 152)	1
  (2, 363)	1
  (2, 311)	1
  (2, 0)	1
  (2, 312)	1
  (2, 44)	1
  (2, 371)	1
  (2, 314)	1
  (2, 46)	1
  (3, 0)	1
  (3, 53)	1
  (3, 353)	1
  (3, 280)	1
  :	:
  (482, 106)	1
  (482, 83)	1
  (482, 12)	1
  (482, 13)	1
  (483, 174)	1
  (483, 353)	1
  (483, 280)	1
  (483, 354)	1
  (483, 388)	1
  (483, 210)	1
  (483, 211)	1
  (483, 303)	1
  (483, 362)	1
  (484, 363)	1
  (484, 174)	1
  (484, 0)	1
  (484, 304)	1
  (484, 388)	1
  (484, 192)	1
  (484, 23)	1
  (484, 386)	1
  (484, 308)	1
  (484, 24)	1
  (484, 194)	1
  (484, 309)	1


In [ ]:
#코사인유사도를 구한 값을 변수에 미리 저장
gerne_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:, ::-1]
gerne_c_sim.shape

(485, 485)

In [ ]:
print(gerne_c_sim)

[[  0  59 440 ... 262 264 216]
 [  1 303 334 ... 178 345 409]
 [  2 154 105 ... 241 129 139]
 ...
 [482 432 332 ... 265 260   0]
 [483 285 363 ... 316 318   0]
 [484 336 416 ... 314 134 211]]


## 함수 구성
1. 입력하는 영화 이름의 값을 타겟으로 설정
2. 타켓 영화와 코사인 유사도로 유사한 정보를 영화를 뽑는다.
3. 유사 영화 리스트에서 평점이 높은 순으로 출력한다.

In [ ]:
#비슷한 영화를 리스트로 반환해주는 함수를 만든다

def get_recommend_movie_list(df, movie_title, top=30):
    # 특정 영화와 비슷한 영화를 추천해야 하기 때문에 '특정 영화' 값를 뽑아낸다.
    target_movie_index = df[df['title'] == movie_title].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = gerne_c_sim[target_movie_index, :top].reshape(-1)
    #본인을 제외
    sim_index = sim_index[sim_index != target_movie_index]

    #data frame으로 만들고 score으로 정렬한 뒤 return
    result = df.iloc[sim_index].sort_values('score', ascending=False)[:5]
    return result

In [ ]:
get_recommend_movie_list(data, movie_title='The Dark Knight Rises')

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
149,155,Drama Action Crime Thriller,8.3,12269.0,123.167259,The Dark Knight,dc comics crime fighter secret identity scarec...,Batman raises the stakes in his war on crime. ...,8.136459
333,106646,Crime Drama Comedy,7.9,6768.0,16.382422,The Wolf of Wall Street,based on novel drug dealer ex-cop rural settin...,A New York stockbroker refuses to cooperate in...,7.702718
420,264644,Drama Thriller,8.1,2838.0,12.443291,Room,NaN,Jack is a young boy of 5 years old who has liv...,7.655355
125,1422,Drama Thriller Crime,7.9,4455.0,18.515448,The Departed,undercover boston police friends mafia underco...,"To take down South Boston's Irish Mafia, the p...",7.626700
325,146233,Drama Thriller Crime,7.9,3183.0,11.962620,Prisoners,prison brother brother relationship drug deale...,When Keller Dover's daughter and her friend go...,7.553214
